In [ ]:
from datetime import datetime
from time import sleep
import requests
from bs4 import BeautifulSoup
import re

from selenium import webdriver
import time

import pandas as pd
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !pip install selenium

In [ ]:
kelompok_usaha = ['hotel',
                  'restoran-rumah-makan',
                  'usaha-golf',
                  'daya-tarik-wisata',
                  'usaha-arung-jeram',
                  'homestay-pondok-wisata',
                  'usaha-wisata-selam',
                  'usaha-jasa-transportasi-wisata']

## scraping selenium

In [ ]:
def scrapeData(url):
    # import the webdriver
    driver = webdriver.Safari()
    driver.get(url)
    def is_next(driver):
        page_items = driver.find_elements_by_xpath(".//li[contains(@class, 'page-item')]")
        classes = [class_name.get_attribute('class') for class_name in page_items]
        if classes[-1] == ' page-item':
            return True
        else:
            return False
    
    is_first = True
    is_next_val = is_next(driver)
    while is_next_val:
        items = []
        containers = driver.find_elements_by_xpath(".//div[@class='card mb-3 px-3 py-0 shadow']")
        for container in containers:
            link_to_desc = container.find_element_by_xpath(".//a").get_attribute("href").strip()
            tipe_usaha = container.find_element_by_xpath(".//h2[@class='h6 text-red font-weight-bold']").text.strip()
            nama_tempat = container.find_element_by_xpath(".//h2[@class='h5 text-blue font-weight-medium ellipsis']").text.strip()
            item_temp = {'link_to_desc':link_to_desc,
                          'tipe_usaha': tipe_usaha,
                          'nama_tempat': nama_tempat}
            items.append(item_temp)
            data_temp = pd.DataFrame(items)
        driver.find_elements_by_xpath(".//li[contains(@class, 'page-item')]")[-1].click()
        if is_first:
            data = data_temp
            is_first = False
        else:
            data = pd.concat([data, data_temp])
        try:
            time.sleep(2)
            is_next_val = is_next(driver)
        except:
            time.sleep(10)
            is_next_val = is_next(driver)

    driver.quit()
    return data

In [ ]:
is_1st = True
for usaha in kelompok_usaha:
    print(usaha)
    url = "https://chse.kemenparekraf.go.id/kategori-usaha/{}".format(usaha)
    data_temp = scrapeData(url)
    if is_1st:
        data_all = data_temp
        is_1st = False
    else:
        data_all = data_all.append(data_temp)
data_all.to_csv('all_fromselenium.csv'.format(usaha))

## Scraping BeautifulSoup

In [ ]:
def get_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'
    }
    try:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        sleep(2)
    except:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        sleep(5)
    return soup

In [ ]:
def get_text(drive):
    try:
        return drive.get_text().strip()
    except:
        return ''

In [ ]:
is_first = True
for usaha in kelompok_usaha:
    print(usaha)
    df = pd.read_csv('all_fromselenium.csv')
    df = df[df['tipe_usaha'] == usaha].reset_index().iloc[:, -3:]
    list_of_d = []
    for i in range(df.shape[0]):
        url = df.loc[i, 'link_to_desc']
        page = get_page(url)
        title = get_text(page.find('h1', class_ ="display-3 font-size-md-down-5 text-white font-weight-semi-bold"))
        no_telp = get_text(page.find('p', class_ ="lead text-white mb-0"))[10:]
        address = get_text(page.find('p', class_ ="lead text-white"))
        items = []
        for side in page.find_all('div', class_ ="col-md-6"):
            for contain in side.find_all('span', class_="text-secondary"):
                items.append(get_text(contain))
        dictionary = {'link_to_dec': df.loc[i, 'link_to_desc'],
                      'tipe_usaha': df.loc[i, 'tipe_usaha'],
                      'nama_tempat': df.loc[i, 'nama_tempat'],
                      'title' : title,
                      'no_telp' : no_telp,
                      'address' : address,
                      'provinsi': items[4],
                      'kota' : items[0],
                      'jenis_usaha' : items[1],
                      'masa_berlaku_mulai' : items[2],
                      'masa_berlaku_sampai' : items[6],
                      'no_sertifikat' : items[3],
                      'kategori_penilaian': items[5]}
        list_of_d.append(dictionary)
    df_temp = pd.DataFrame(list_of_d)
    if is_first:
        df_all = df_temp
        is_first = False
    else:
        df_all = df_all.append(df_temp)
    df_all.to_csv("data_all.csv")

In [ ]:
pd.read_csv("data_all.csv").shape

# Hit Lat Long

In [ ]:
KEY = "INSERT_YOUR_SECRET_KEY"
df_all = pd.read_csv('data_chse.csv').iloc[:,1:]

In [ ]:
# df_all.address[0]

In [ ]:
df = df_all.copy()
df['address_components'] = np.nan
df['address_lenght'] = np.nan
df['formatted_address'] = np.nan
df['lat'] = np.nan
df['lon'] = np.nan
df['address_types'] = np.nan
df['place_id'] = np.nan

for i in range(df.shape[0]):
    address = re.sub(r'\n|[^a-zA-Z .,0-9]', "", df.loc[i, 'address'])
    city = df.loc[i, 'kota']
    province = df_all.loc[i, 'provinsi']
    addr_full = "{}, {}, {}".format(address, city, province)
    url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(addr_full, KEY)
    result_ = requests.get(url).json()
    if result_['status'] == 'OK':
        result = result_['results'][0]
        df['address_components'][i] = result['address_components']
        df['address_lenght'][i] = len(result['address_components'])
        df['formatted_address'][i] = result['formatted_address']
        df['lat'][i] = result['geometry']['location']['lat']
        df['lon'][i] = result['geometry']['location']['lng']
        df['address_types'][i] = result['types']
        df['place_id'][i] = result['place_id']
    else:
        addr_full = re.sub(r'\n|[^a-zA-Z .,0-9]', "", df.loc[i, 'address'])
        url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(addr_full, KEY)
        result_ = requests.get(url).json()
        if result_['status'] == 'OK':
            result = result_['results'][0]
            df['address_components'][i] = result['address_components']
            df['address_lenght'][i] = len(result['address_components'])
            df['formatted_address'][i] = result['formatted_address']
            df['lat'][i] = result['geometry']['location']['lat']
            df['lon'][i] = result['geometry']['location']['lng']
            df['address_types'][i] = result['types']
            df['place_id'][i] = result['place_id']
        else:
            addr_full = "{}, {}".format(city, province)
            url = "https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}".format(addr_full, KEY)
            result_ = requests.get(url).json()
            if result_['status'] == 'OK':
                result = result_['results'][0]
                df['address_components'][i] = result['address_components']
                df['address_lenght'][i] = len(result['address_components'])
                df['formatted_address'][i] = result['formatted_address']
                df['lat'][i] = result['geometry']['location']['lat']
                df['lon'][i] = result['geometry']['location']['lng']
                df['address_types'][i] = result['types']
                df['place_id'][i] = result['place_id']
            else:
                print(result_['status'], addr_full, i)
df.to_csv("data_chse_w_lonlat.csv")

In [ ]:
df.shape

(7924, 20)

In [ ]:
df = pd.read_csv("data_chse_w_lonlat.csv").iloc[:, 1:]
df['no_telp'] = df['no_telp'].astype('int64')
df.to_pickle("data_chse_w_lonlat.pkl")

In [ ]:
from fuzzywuzzy import fuzz

/Users/ilhamrizky/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
!pip install fuzzywuzzy

In [ ]:
Str1 = "Los Angeles Lakers"
Str2 = "Lakers"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
print(Ratio)
print(Partial_Ratio)

50
100
